# Reproducing Fig. 1b

This notebook examplifies how to reproduce figure 1b of the article. the annotations from all screens will be downloaded and parsed to build statistics on phenotypes, which will eb displayed using bokeh.

## Preamble

In [152]:
from pandas import DataFrame,concat
import omero
import numpy as np

import matplotlib.pyplot as plt
from bokeh.models import ColumnDataSource
from bokeh.plotting import figure, output_notebook, show, save
from bokeh.charts import Scatter
from bokeh.models.formatters import TickFormatter, String, List, Dict, Int
from bokeh.models import FixedTicker,HoverTool

output_notebook()
%matplotlib inline

ImportError: No module named bokeh.models

In [2]:
from idr_demo import connection

conn = connection()

In [1]:
def getBulkAnnotationAsDf(screenID,conn):
#    ofId=8118685
    sc=conn.getObject('Screen',screenID)
    for ann in sc.listAnnotations():
        if isinstance(ann, omero.gateway.FileAnnotationWrapper):
            if (ann.getFile().getName()=='bulk_annotations'):
                if (ann.getFile().getSize()> 147625090): #about 140Mb?
                    print "that's a big file..."
                    return None
                ofId=ann.getFile().getId()
                break


    original_file = omero.model.OriginalFileI(ofId, False)

    openTable = conn.c.sf.sharedResources().openTable(original_file)
    rowCount = openTable.getNumberOfRows()

    #table to df

    column_names = [col.name for col in openTable.getHeaders()]

    black_list = []
    column_indices = []
    for column_name in column_names:
        if column_name in black_list:
            continue
        column_indices.append(column_names.index(column_name))

    table_data = openTable.slice(column_indices, None)
    data = []
    for index in range(rowCount):
        row_values = [column.values[index] for column in table_data.columns]
        data.append(row_values)

    dfAnn=DataFrame(data)
    dfAnn.columns=column_names
    return dfAnn

In [27]:
def appendPhInfo(phall,screen,df):
# extract all phenotypes informatino from given bulk annotation file and
# append it to the phall
    phcol=df.columns[[('Phenotype' in s) and ('Term Accession' in s) for s in df.columns ]]
    for s in phcol:
        ph=df[s].unique()
        if ph[0]!='':
            ph=ph[0]            
            desc=df[s.replace('Accession', 'Name')].unique()[0]
        else:
            ph=ph[1]
            desc=df[s.replace('Accession', 'Name')].unique()[1]
        dfph=df[df[s]!='']
        try:
            phall[ph]['n']=phall[ph]['n']+len(dfph)
            if not (screen in phall[ph]['screens']): 
                phall[ph]['screens'].append(screen)

        except Exception as e: 
            print(e)
            phcur={'n':len(dfph),'screens':[screen],'desc':desc}
            phall[ph]=phcur
    

## Build and display figure

In [147]:

#screens=[3,206,1202]
screens= conn.getObjects("Screen")

phall={}
for sc in screens:
    scId=sc.getId()
    name=sc.getName()
    print 'loading '+str(scId)+'::'+name
    try:
        df=getBulkAnnotationAsDf(scId,conn)
        appendPhInfo(phall,name,df)
    except:
        print 'fail!'

loading 3::idr0001-graml-sysgro/screenA
'PATO_0000460'
'CMPO_0000369'
'CMPO_0000370'
'CMPO_0000371'
'CMPO_0000372'
'CMPO_0000365'
'CMPO_0000077'
'CMPO_0000367'
'CMPO_0000116'
'CMPO_0000364'
'CMPO_0000366'
'CMPO_0000118'
'GO_0022403'
'CMPO_0000383'
'CMPO_0000388'
'CMPO_0000378'
'CMPO_0000387'
'CMPO_0000412'
'CMPO_0000410'
'CMPO_0000413'
loading 102::idr0002-heriche-condensation/screenA
'CMPO_0000329'
'CMPO_0000328'
loading 51::idr0003-breker-plasticity/screenA
'CMPO_0000391'
'CMPO_0000392'
'CMPO_0000393'
'CMPO_0000394'
'CMPO_0000395'
'CMPO_0000396'
'CMPO_0000397'
'CMPO_0000398'
'CMPO_0000400'
'CMPO_0000401'
'CMPO_0000402'
loading 202::idr0004-thorpe-rad52/screenA
'CMPO_0000415'
loading 253::idr0006-fong-nuclearbodies/screenA
'CMPO_0000404'
'CMPO_0000405'
'CMPO_0000406'
'CMPO_0000407'
'CMPO_0000408'
'CMPO_0000409'
loading 201::idr0007-srikumar-sumo/screenA
'PATO_0000462'
loading 154::idr0008-rohn-actinome/screenA
'CMPO_0000129'
'CMPO_0000128'
'CMPO_0000270'
'CMPO_0000267'
'CMPO_0000274'


In [150]:
TOOLS="pan,wheel_zoom,reset"

p = figure(title = "Fig 1b", tools=TOOLS,y_axis_type="log",width=1200,toolbar_location="above")


source = ColumnDataSource(
    data=dict(
        ph=phall.keys(),
#        mid=[c['gfp_mid_enrich_vs_tip'] for c in cells],
        n=[phall[ph]['n'] for ph in phall.keys()],
        names=[phall[ph]['screens'] for ph in phall.keys()],
        desc=[phall[ph]['desc'] for ph in phall.keys()],
#        mid=[c['gfp_mid_enrich'] for c in cells],
        x=[2*x for x in range(len(phall.keys()))],
        r=[1*len(phall[ph]['screens']) for ph in phall.keys()]
    ))

label_data = {2*i:ph for i,x in enumerate(phall.keys())}

cir=p.circle('x','n',radius='r',source=source)


hover = HoverTool(
        tooltips=[
            ("Term", "@ph"),
            ("Description", "@desc"),
            ("Number of samples", "@n"),
            ("Screens name", "@names"),
        ]
    )
p.add_tools(hover)

JS_CODE =  """
        _ = require "underscore"
        Model = require "model"
        p = require "core/properties"
        class FixedTickFormatter extends Model
          type: 'FixedTickFormatter'
          doFormat: (ticks) ->
            labels = @get("labels")
            return (labels[tick] ? "" for tick in ticks)
          @define {
            labels: [ p.Any ]
          }
        module.exports =
          Model: FixedTickFormatter
    """

class FixedTickFormatter(TickFormatter):

    labels = Dict(Int, String, help="""
    A mapping of integer ticks values to their labels.
    """)

    __implementation__ = JS_CODE


p.xaxis.formatter = FixedTickFormatter(labels=label_data)
p.xaxis.ticker = FixedTicker(ticks=sorted(label_data.keys()))
p.xaxis.major_label_orientation = np.pi/4.
p.xaxis.axis_label_text_font_size = "10pt"
show(p)
save(p,'fig1b.html')

NameError: name 'figure' is not defined

## export data

In [97]:
import pickle


In [149]:
output = open('phall.pkl', 'wb')
pickle.dump(phall,output)
output.close()